# Pydantic + HuggingFace + Biopython 통합 실습

**학습목표**: Biopython으로 PubMed에서 논문 초록을 가져와서 HuggingFace API와 Pydantic을 사용해 구조화된 분석을 수행한다.

**구성요소**:
- Biopython Entrez: PubMed 데이터 수집
- Pydantic: 데이터 구조 정의 및 검증
- HuggingFace: LLM 기반 텍스트 분석"

### Huggingface 예제 코드
[예제 링크](https://huggingface.co/docs/inference-providers/guides/structured-output)

In [44]:
# https://huggingface.co/docs/inference-providers/guides/responses-api

import os
from openai import OpenAI
from dotenv import load_dotenv
from huggingface_hub import InferenceClient
from pydantic import BaseModel

load_dotenv()
# Initialize the client
client = InferenceClient(
    provider="cerebras",  # or use "auto" for automatic selection
    api_key=os.environ["HF_TOKEN"],
)

# Example paper text (truncated for brevity)
paper_text = """
Title: Attention Is All You Need

Abstract: The dominant sequence transduction models are based on complex recurrent 
or convolutional neural networks that include an encoder and a decoder. The best 
performing models also connect the encoder and decoder through an attention mechanism. 
We propose a new simple network architecture, the Transformer, based solely on 
attention mechanisms, dispensing with recurrence and convolutions entirely...
"""

# Define the response format
class PaperAnalysis(BaseModel):
    title: str
    abstract_summary: str

# Convert the Pydantic model to a JSON Schema and wrap it in a dictionary
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "PaperAnalysis",
        "schema": PaperAnalysis.model_json_schema(),
        "strict": True,
    },
}

# Define your messages with a system prompt and a user prompt
# The system prompt is a description of the task you want the model to perform
# The user prompt is the input data you want to process
messages = [
    {
        "role": "system", 
        "content": "Extract paper title and abstract summary."
    },
    {
        "role": "user", 
        "content": paper_text
    }
]

# Generate structured output using Qwen/Qwen3-32B model
response = client.chat_completion(
    messages=messages,
    response_format=response_format,
    model="Qwen/Qwen3-32B",
)

# The response is guaranteed to match your schema
structured_data = response.choices[0].message.content
print(structured_data)

{"title": "Attention Is All You Need", "abstract_summary": "This paper demonstrates how the proposed Transformer model eliminates complex recurrent and convolutional neural networks while maintaining the encoder-decoder structure with attention mechanisms. The analysis showcases a new architecture based solely on attention mechanisms that outperforms traditional models in sequence transduction tasks."}


In [2]:
# 필요한 라이브러리 import
import os
from dotenv import load_dotenv
from Bio import Entrez
from huggingface_hub import InferenceClient
from pydantic import BaseModel, Field
from typing import List, Optional
import json

# 환경 변수 로드
load_dotenv()

# HuggingFace 클라이언트 초기화
client = InferenceClient(
    provider="cerebras",
    api_key=os.environ["HF_TOKEN"],
)

print(f"Entrez Email: {Entrez.email}")
print(f"HF API Key 설정: {bool(os.environ.get('HF_TOKEN'))}")


Entrez Email: None
HF API Key 설정: True


/Users/insilicogen/kbiohealth_교육/Python-Advanced/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pydantic 모델 정의

In [36]:
# PubMed 논문 정보를 담는 Pydantic 모델
class PubMedArticle(BaseModel):
    pmid: str
    title: str
    abstract: str
    authors: List[str]
    journal: str
    pub_year: Optional[str] = None

# HuggingFace로 분석할 구조화된 출력 모델
class PaperAnalysis(BaseModel):
    """논문 분석 결과를 구조화하는 모델"""
    title: str = Field(description="논문 제목")
    abstract_summary: str = Field(description="초록 요약 (2-3문장)")
    key_findings: List[str] = Field(description="주요 발견사항 목록")
    methodology: str = Field(description="사용된 연구 방법론")
    significance: str = Field(description="연구의 중요성 및 영향")
    keywords: List[str] = Field(description="핵심 키워드 5개 이하")

print(f"PubMedArticle 스키마: {PubMedArticle.model_json_schema()}")
print(f"\nPaperAnalysis 스키마: {PaperAnalysis.model_json_schema()}")


PubMedArticle 스키마: {'properties': {'pmid': {'title': 'Pmid', 'type': 'string'}, 'title': {'title': 'Title', 'type': 'string'}, 'abstract': {'title': 'Abstract', 'type': 'string'}, 'authors': {'items': {'type': 'string'}, 'title': 'Authors', 'type': 'array'}, 'journal': {'title': 'Journal', 'type': 'string'}, 'pub_year': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Pub Year'}}, 'required': ['pmid', 'title', 'abstract', 'authors', 'journal'], 'title': 'PubMedArticle', 'type': 'object'}

PaperAnalysis 스키마: {'description': '논문 분석 결과를 구조화하는 모델', 'properties': {'title': {'description': '논문 제목', 'title': 'Title', 'type': 'string'}, 'abstract_summary': {'description': '초록 요약 (2-3문장)', 'title': 'Abstract Summary', 'type': 'string'}, 'key_findings': {'description': '주요 발견사항 목록', 'items': {'type': 'string'}, 'title': 'Key Findings', 'type': 'array'}, 'methodology': {'description': '사용된 연구 방법론', 'title': 'Methodology', 'type': 'string'}, 'significance': {'descriptio

### Biopython으로 PubMed 데이터 수집

In [18]:
def fetch_pubmed_articles(search_term: str, max_results: int = 5) -> List[PubMedArticle]:
    """PubMed에서 논문을 검색하고 PubMedArticle 객체 리스트로 반환"""
    articles = []
    
    try:
        # 1. PubMed에서 검색
        print(f"'{search_term}' 검색 중...")
        handle = Entrez.esearch(db="pubmed", term=search_term, retmax=max_results)
        search_results = Entrez.read(handle)
        handle.close()
        
        pmids = search_results['IdList']
        print(f"{len(pmids)}개 논문 발견")
        
        if not pmids:
            return articles
        
        # 2. 각 논문의 상세 정보 가져오기
        handle = Entrez.efetch(db="pubmed", id=pmids, rettype="abstract", retmode="xml")
        records = Entrez.read(handle)['PubmedArticle']
        
        for record in records:
            try:
                # 논문 정보 추출
                article_info = record['MedlineCitation']['Article']
                
                # PMID
                pmid = record['MedlineCitation']['PMID']
                
                # 제목
                title = article_info.get('ArticleTitle', 'No title')
                
                # 초록
                abstract_text = ""
                if 'Abstract' in article_info:
                    abstract = article_info['Abstract']
                    if 'AbstractText' in abstract:
                        abstract_sections = abstract['AbstractText']
                        if isinstance(abstract_sections, list):
                            # 구조화된 초록
                            abstract_parts = []
                            for section in abstract_sections:
                                if isinstance(section, dict):
                                    label = section.get('@Label', '')
                                    text = section.get('#text', str(section))
                                    if label:
                                        abstract_parts.append(f"{label}: {text}")
                                    else:
                                        abstract_parts.append(text)
                                else:
                                    abstract_parts.append(str(section))
                            abstract_text = " ".join(abstract_parts)
                        else:
                            abstract_text = str(abstract_sections)
                
                # 저자들
                authors = []
                if 'AuthorList' in article_info:
                    for author in article_info['AuthorList']:
                        if isinstance(author, dict):
                            if 'LastName' in author and 'ForeName' in author:
                                authors.append(f"{author['ForeName']} {author['LastName']}")
                            elif 'CollectiveName' in author:
                                authors.append(author['CollectiveName'])
                
                # 저널
                journal = article_info.get('Journal', {}).get('Title', 'Unknown journal')
                
                # 발행연도
                pub_year = None
                if 'Journal' in article_info and 'JournalIssue' in article_info['Journal']:
                    pub_date = article_info['Journal']['JournalIssue'].get('PubDate', {})
                    pub_year = pub_date.get('Year')
                
                # PubMedArticle 객체 생성
                article = PubMedArticle(
                    pmid=str(pmid),
                    title=title,
                    abstract=abstract_text,
                    authors=authors[:5],  # 최대 5명의 저자만
                    journal=journal,
                    pub_year=pub_year
                )
                
                articles.append(article)
                print(f"PMID {pmid}: {title[:50]}...")
                
            except Exception as e:
                print(f"논문 파싱 오류: {e}")
                continue
        
        handle.close()
        
    except Exception as e:
        print(f"PubMed 검색 오류: {e}")
    
    return articles

# 실제 검색 수행
search_query = '"machine learning" AND bioinformatics AND 2024[DP]'
pubmed_articles = fetch_pubmed_articles(search_query, max_results=3)

print(f"\n수집 완료: {len(pubmed_articles)}개 논문")
for i, article in enumerate(pubmed_articles):
    print(f"\n{i+1}. PMID: {article.pmid}")
    print(f"   제목: {article.title[:80]}...")
    print(f"   저자: {', '.join(article.authors[:3])}")
    print(f"   저널: {article.journal}")
    print(f"   초록 길이: {len(article.abstract)} 문자")


'"machine learning" AND bioinformatics AND 2024[DP]' 검색 중...
3개 논문 발견
PMID 41031031: Rapid prediction of thermodynamically destabilizin...
PMID 40995405: Persistent Laplacian-enhanced algorithm for scarce...
PMID 40979399: Editorial: AI/ML in pharmacovigilance and pharmaco...

수집 완료: 3개 논문

1. PMID: 41031031
   제목: Rapid prediction of thermodynamically destabilizing tyrosine phosphorylations in...
   저자: Jaie Woodard, Zhengqing Liu, Atena Malemir Chegini
   저널: bioRxiv : the preprint server for biology
   초록 길이: 1879 문자

2. PMID: 40995405
   제목: Persistent Laplacian-enhanced algorithm for scarcely labeled data classification...
   저자: Gokul Bhusal, Ekaterina Merkurjev, Guo-Wei Wei
   저널: Machine learning
   초록 길이: 1732 문자

3. PMID: 40979399
   제목: Editorial: AI/ML in pharmacovigilance and pharmacoepidemiology....
   저자: Wen Zou, Pantelis Natsiavas, Assaf Gottlieb
   저널: Frontiers in drug safety and regulation
   초록 길이: 0 문자


### HuggingFace로 구조화된 논문 분석"

In [ ]:
def analyze_paper_with_hf(article: PubMedArticle) -> PaperAnalysis:
    """HuggingFace API를 사용해 논문을 구조화된 형태로 분석"""
    
    # Pydantic 모델을 JSON Schema로 변환
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "name": "PaperAnalysis",
            "schema": PaperAnalysis.model_json_schema(),
            "strict": True,
        },
    }
    
    # 논문 텍스트 준비
    paper_text = f"""
    Title: {article.title}
    
    Authors: {', '.join(article.authors)}
    
    Journal: {article.journal} ({article.pub_year})
    
    Abstract: {article.abstract}
    """
    
    # 시스템 프롬프트와 사용자 프롬프트 정의
    messages = [
        {
            "role": "system", 
            "content": """
            당신은 생물정보학 및 기계학습 분야의 전문가입니다. 
            주어진 논문을 분석하여 다음 항목들을 추출하세요:
            
            1. 논문 제목 (정확히)
            2. 초록 요약 (2-3문장으로 핵심 내용)
            3. 주요 발견사항 (구체적인 결과들)
            4. 연구 방법론 (사용된 기법이나 접근법)
            5. 연구의 중요성 및 영향
            6. 핵심 키워드 (5개 이하)
            
            모든 응답은 한국어로 작성하고, 구조화된 JSON 형태로 반환하세요.
            """
        },
        {
            "role": "user", 
            "content": paper_text
        }
    ]
    
    try:
        # HuggingFace API 호출
        print(f"PMID {article.pmid} 분석 중...")
        response = client.chat_completion(
            messages=messages,
            response_format=response_format,
            model="Qwen/Qwen3-32B",
            max_tokens=3000,
            temperature=0.7
        )
        # JSON 응답 파싱
        structured_output = response.choices[0].message.content
        analysis_data = json.loads(structured_output)
        # Pydantic 모델로 검증 및 변환
        analysis = PaperAnalysis(**analysis_data)
        
        print(f"분석 완료: {analysis.title[:50]}...")
        return analysis
        
    except Exception as e:
        print(f"분석 오류: {e}")
        # 기본값 반환
        return PaperAnalysis(
            title=article.title,
            abstract_summary="분석 실패",
            key_findings=["분석을 완료할 수 없습니다"],
            methodology="알 수 없음",
            significance="분석 실패",
            keywords=["오류"]
        )

# 첫 번째 논문 분석
if pubmed_articles:
    first_article = pubmed_articles[0]
    print(f"\n분석할 논문: {first_article.title[:60]}...")
    print(f"초록 미리보기: {first_article.abstract[:200]}...\n")
    
    analysis_result = analyze_paper_with_hf(first_article)
    
    print("\n=== 분석 결과 ===")
    print(f"제목: {analysis_result.title}")
    print(f"\n요약: {analysis_result.abstract_summary}")
    print(f"\n주요 발견사항:")
    for i, finding in enumerate(analysis_result.key_findings, 1):
        print(f"  {i}. {finding}")
    print(f"\n방법론: {analysis_result.methodology}")
    print(f"\n중요성: {analysis_result.significance}")
    print(f"\n키워드: {', '.join(analysis_result.keywords)}")
else:
    print("분석할 논문이 없습니다.")



분석할 논문: Rapid prediction of thermodynamically destabilizing tyrosine...
초록 미리보기: Tyrosine phosphorylations are a prominent characteristic of numerous cancers, necessitating the use of computational tools to comprehensively analyze phosphoproteomes and identify potentially (dys)fun...

🤖 PMID 41031031 분석 중...
분석 완료: Rapid prediction of thermodynamically destabilizin...

=== 분석 결과 ===
제목: Rapid prediction of thermodynamically destabilizing tyrosine phosphorylations in cancers

요약: 이 논문은 타이로신 인산화의 열역학적 안정도 감소를 예측하는 머신러닝 기반 방법을 제안한다. AlphaFold2 데이터베이스를 이용해 384,857개의 타이로신 residue를 분석하고, 600개 이상의 고유 인산화 유형을 포함한 다종양 데이터를 적용하여 자가억제 또는 활성화 구조 변화와 연관된 인산화를 예측하였다. 생물학적 경로와 네트워크 중심성과의 상관관계도 밝혔다.

주요 발견사항:
  1. pKm (인산모이테크 델타-델타-G) 예측 모델의 실험적 상관계수 R=0.71 달성
  2. 자가억제 온코젠의 인산화 상태 예측 AUC 0.85
  3. 화학적/산화적 스트레스 경로에서 최상위 인산화 단백질의 풍부화
  4. 대사 단백질에서 네트워크 중심성 낮은 단백질의 인산화 소멸 경향 (p=0.005)
  5. 종양 재발 인산화 중 58%가 1 kcal/mol 기준에서 불안정화

방법론: 극단적 그라디언트 부스팅 머신러닝 접근법을 사용하여 구조적 특성과 회로 위상 병렬 관계를 기반으로 인산화 유도 불안정화를 예측

In [42]:
print(analysis_result.model_dump_json(indent=2))


{
  "title": "Rapid prediction of thermodynamically destabilizing tyrosine phosphorylations in cancers",
  "abstract_summary": "이 논문은 타이로신 인산화의 열역학적 안정도 감소를 예측하는 머신러닝 기반 방법을 제안한다. AlphaFold2 데이터베이스를 이용해 384,857개의 타이로신 residue를 분석하고, 600개 이상의 고유 인산화 유형을 포함한 다종양 데이터를 적용하여 자가억제 또는 활성화 구조 변화와 연관된 인산화를 예측하였다. 생물학적 경로와 네트워크 중심성과의 상관관계도 밝혔다.",
  "key_findings": [
    "pKm (인산모이테크 델타-델타-G) 예측 모델의 실험적 상관계수 R=0.71 달성",
    "자가억제 온코젠의 인산화 상태 예측 AUC 0.85",
    "화학적/산화적 스트레스 경로에서 최상위 인산화 단백질의 풍부화",
    "대사 단백질에서 네트워크 중심성 낮은 단백질의 인산화 소멸 경향 (p=0.005)",
    "종양 재발 인산화 중 58%가 1 kcal/mol 기준에서 불안정화"
  ],
  "methodology": "극단적 그라디언트 부스팅 머신러닝 접근법을 사용하여 구조적 특성과 회로 위상 병렬 관계를 기반으로 인산화 유도 불안정화를 예측. AlphaFold2 데이터베이스를 활용한 대규모 타이로신 residue 분석 및 11종 종양의 인산화 데이터셋 적용.",
  "significance": "암에서의 인산화 조절 메커니즘 이해를 가속화하고, 자가억제 단백질의 활성화 경로를 규명하여 약물 개발 기반을 제공. 대규모 인산화 변이 및 불안정화 단백질의 신속한 스크리닝 가능성을 제시.",
  "keywords": [
    "머신러닝",
    "타이로신 인산화",
    "열역학적 안정도",
    "암",
    "인산화 단백질"
  ]
}


## 🤗수고 많으셨습니다🤗